
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Creating a Pseudonymized PII Lookup Tables

In this lesson we'll create a pseudonymized key for storing potentially sensitive user data.  
Our approach in this notebook is fairly straightforward; some industries may require more elaborate de-identification to guarantee privacy.

We'll examine design patterns for ensuring PII is stored securely and updated accurately. 

##### Objectives
- Describe the purpose of "salting" before hashing
- Apply salted hashing to sensitive data(user_id)
- Apply tokenization to sensitive data(user_id)

##### Creates the Following
  1. **registered_users** table from the source JSON files with PII.

  1. Hashing: Handled in table **user_lookup_hashed**

  1. Tokenization: Handled in tables **registered_users_tokens** and **user_lookup_tokenized**

### A. DAG


![demo01_2_pii_data_security_pseudo_dag.png](../Includes/images/demo01_2_pii_data_security_pseudo_dag.png)

In [0]:
from pyspark import pipelines as dp
import pyspark.sql.functions as F

user_reg_source = spark.conf.get("user_reg_source")

## B. Registered Users

This is our source data to apply

In [0]:
# Ingest data into the registered_users table incrementally with Auto Loader
@dp.table
def registered_users():
    return (
        spark.readStream
            .format("cloudFiles")
            .schema("device_id LONG, mac_address STRING, registration_timestamp DOUBLE, user_id LONG")
            .option("cloudFiles.format", "json")
            .load(f"{user_reg_source}")
        )


## C. Create pseudonymized user lookup table


### C1. with salted hashing

Create a function to register this logic to the current database under the name **`salted_hash`**. This will allow this logic to be called by any user with appropriate permissions on this function. 

Note that it is theoretically possible to link the original key and pseudo-ID if the hash function and the salt are known. Here, we use this method to add a layer of obfuscation; in production, you may wish to have a much more sophisticated hashing method.

In [0]:
salt = "BEANS"
     
# Define function to pseudonymize with salted hashing    
def salted_hash(id):
    return F.sha2(F.concat(id, F.lit(salt)), 256)


The logic below creates the **user_lookup** table. In the next notebook, we'll use this pseudo-ID as the sole link to user PII. By controlling access to the link between our **alt_id** and other natural keys, we'll be able to prevent linking PII to other user data throughout our system.

Use the function above to create the **alt_id** to the **user_id** from the **registered_users** table. Make sure to include all necessary columns for the target **user_lookup** table.

In [0]:
# Create pseudonymized user lookup table
# Method: Hashing
@dp.table
def user_lookup_hashed():
    return (dp
            .read_stream("registered_users")
            .select(
                  salted_hash(F.col("user_id")).alias("alt_id"),
                  "device_id", "mac_address", "user_id")
           )

### C2. with Tokenization

Lets first create a table that will hold the tokens for our users in **registered_token** table

In [0]:
@dp.table
def registered_users_tokens():
    return (dp
            .readStream("registered_users")
            .select("user_id")
            .distinct()
            .withColumn("token", F.expr("uuid()"))
        )


Now lets create the table **user_lookup_tokenized** using the **registered_users_tokens** and have a join to include the new tokenized column as **alt_id**.


In [0]:
# Create pseudonymized user lookup table
# Method: Tokenization
@dp.table
def user_lookup_tokenized():
    return (dp
            .read_stream("registered_users")
            .join(dp.read("registered_users_tokens"), "user_id", "left")
            .drop("user_id")
            .withColumnRenamed("token", "alt_id")
           )

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>